In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 10 09:47:18 2023

@author: Administrator
"""
import random
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils import data
import train_fun as tf
import data_prepare as dpp
import time

'''
all_lab=[]
labs=[0.1,0.2,0.4,0.8,1.6,12.0,25.0,3.0,6.0]
for i in range(9):
    labs[i]=labs[i]*0.568
    for j in range(1000):
        all_lab.append(labs[i])
all_lab=np.array(all_lab)
all_lab=all_lab.reshape(-1,1)
pd.DataFrame(all_lab).to_csv('E:/实验数据/数据处理后文件/20221216机器学习结果/LIBS_data/all_lab.csv',header=None,index=False)
'''

###########################实验数据读取######################################################
all_lab = pd.read_csv('LIBS_data/all_lab.csv', header=None).values.astype(np.float32)
wave = pd.read_table('LIBS_data/1_000.txt', header=None, sep=';').values.astype(np.float32)
wave = wave[:, 0]
all_feat = pd.read_csv('LIBS_data/all_feat.csv', header=None).values.astype(np.float32)
all_lab = all_lab * 50

###########################光谱平均########################################################
step_avg = 100  # 平均的时候为100，不平均的话改成1
num_sample = 9
avg_feat = dpp.spec_avg_normal(all_feat, step_avg)
avg_lab = dpp.spec_avg_normal(all_lab, step_avg)
sample_delet = [6]
avg_feat, avg_lab = dpp.delet_sample(avg_feat, avg_lab, sample_delet, num_sample)
Cr_lab = avg_lab
Fe_lab = Cr_lab*2.1037
#spec_num = (len(avg_lab) // num_sample)
#avg_feat = np.delete(avg_feat, np.arange(6 * spec_num, 7 * spec_num), axis=0)
#avg_lab = np.delete(avg_lab, np.arange(6 * spec_num, 7 * spec_num), axis=0)
print('光谱平均完成')
###########################归一化及特征选择########################################################
avg_feat_normal = dpp.normal_pixel(avg_feat)
scores = dpp.score(avg_feat_normal, Fe_lab)  # 做Cr的时候是Cr_lab,做Fe的时候改成Fe_lab
num_feat = 150
index_feat = dpp.select_best(scores, num_feat)
select_feat = avg_feat_normal[:, index_feat]
select_wave = wave[index_feat]
select_scores = scores[index_feat]
pd.DataFrame(select_wave).to_csv('select_wave_Fe.csv', header=False, index=False)  # 做Fe的时候记得Cr修改为Fe
pd.DataFrame(select_scores).to_csv('select_score_Fe.csv', header=False, index=False)
pd.DataFrame(select_feat).to_csv('select_feat_Fe.csv', header=False, index=False)
print('归一化及特征选择完成')

###########################数据集划分及包装####################################################
batch_size = 64  # 平均的时候为64，不平均则为4096
index_test = [1, 7]
train_feat, train_lab, test_feat, test_lab = tf.stand_divid(select_feat, Fe_lab, 8, index_test)
train_feat = tf.a2t(train_feat)
train_lab_tensor = tf.a2t(train_lab)
test_feat = tf.a2t(test_feat)
test_lab_tensor = tf.a2t(test_lab)
train_data = tf.load_array((train_feat, train_lab_tensor), batch_size)
###########################模型训练及保存####################################################
model = tf.bpnn()
model = tf.model_train(model, train_data, train_feat, train_lab_tensor, test_feat, test_lab_tensor)
torch.save(model, 'model_Fe.pth')  # 记得修改
###########################模型性能评估####################################################
train_pred_lab = model(train_feat)
test_pred_lab = model(test_feat)
train_pred_lab = (train_pred_lab.detach().numpy())
test_pred_lab = (test_pred_lab.detach().numpy())
slope = dpp.compute_slope(train_lab, train_pred_lab)
r2 = dpp.compute_r2(train_lab, train_pred_lab)
rec = dpp.compute_MRE(train_lab, train_pred_lab)
rep = dpp.compute_MRE(test_lab, test_pred_lab)
rsd = dpp.compute_rsd(test_pred_lab, 2)
model_performance = [slope, r2, rec, rep, rsd]
model_performance = np.array(model_performance)
# all_model_performance.append(model_performance)
pd.DataFrame(model_performance).to_csv('model_performance_Fe.csv', header=False, index=False)
###########################校准曲线绘图####################################################
train_lab = train_lab / 50
test_lab = test_lab / 50
train_pred_lab = train_pred_lab / 50
test_pred_lab = test_pred_lab / 50
mean_sd = dpp.get_sd_mean(train_lab, train_pred_lab, test_lab, test_pred_lab, 8)
pd.DataFrame(mean_sd).to_csv('mean_sd_Fe.csv', header=False, index=False)


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Feb 22 20:56:04 2023

@author: zhangzhao
"""

import random
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils import data
import train_fun as tf
import data_prepare as dpp
import time

##################################数据载入#####################################################
all_lab = pd.read_csv('LIBS_data/all_lab.csv', header=None).values.astype(np.float32)
# chunk_size = 3000
# all_feat = dpp.load_big_csv('LIBS_data/avg_feat', chunk_size)
all_feat = pd.read_csv('LIBS_data/all_feat.csv', header=None).values.astype(np.float32)
all_lab = all_lab * 50
print('数据读取完成')
####################################数据归一化##############################################
all_feat_normal = dpp.normal_pixel(all_feat)
print('归一化完成')
###################################进行随机采样#################################################
avg_num = 0
avg_step = 2
all_model_performance = []
num_sample = 9
num_spec = len(all_feat) // num_sample
all_index = list(np.arange(len(all_feat)))
for i in range(50):
    avg_num = avg_num + avg_step
    print('%d张平均开始' % avg_num)
    index_random = []
    for j in range(num_sample):
        index = random.sample(all_index[j * num_spec:(j + 1) * num_spec], avg_num * 10)
        index = np.array(index)
        index_random.append(index)
    index_random = np.array(index_random)
    index_random = index_random.ravel()
    random_feat = all_feat_normal[index_random, :]
    random_lab = all_lab[index_random, :]
    random_lab = random_lab.reshape(-1, 1)
    ###############################光谱平均处理及特征选择#####################################################
    avg_feat = dpp.spec_avg_normal(random_feat, avg_num)
    avg_lab = dpp.spec_avg_normal(random_lab, avg_num)
    scores = dpp.score(avg_feat, avg_lab)
    num_feat = 150
    index_feat = dpp.select_best(scores, num_feat)
    select_feat = avg_feat[:, index_feat]
    ##############################数据集划分及包装##########################################################
    batch_size = 64
    index_test = [1, 5]
    train_feat, train_lab, test_feat, test_lab = tf.stand_divid(select_feat, avg_lab, 9, index_test)
    train_feat = tf.a2t(train_feat)
    train_lab_tensor = tf.a2t(train_lab)
    test_feat = tf.a2t(test_feat)
    test_lab_tensor = tf.a2t(test_lab)
    train_data = tf.load_array((train_feat, train_lab_tensor), batch_size)
    ##############################模型训练##################################################################
    print('模型训练开始')
    model = tf.bpnn()
    model = tf.model_train(model, train_data, train_feat, train_lab_tensor, test_feat, test_lab_tensor)
    print('模型训练结束')
    #############################模型性能评估################################################################
    train_pred_lab = model(train_feat).detach().numpy()
    test_pred_lab = model(test_feat).detach().numpy()
    r2 = dpp.compute_r2(train_lab, train_pred_lab)
    slope = dpp.compute_slope(train_lab, train_pred_lab)
    rec = dpp.compute_MRE(train_lab, train_pred_lab)
    rep = dpp.compute_MRE(test_lab, test_pred_lab)
    rsd = dpp.compute_rsd(test_pred_lab, 2)
    model_performance = np.array([r2, slope, rec, rep, rsd])
    pd.DataFrame(model_performance).to_csv('model_performance_' + str(avg_num) + '.csv', index=False, header=False)
    print('模型性能评估完成')
    all_model_performance.append(model_performance)
    ############################获取均值与标准差#####################################################################
    sd_mean = dpp.get_sd_mean(train_lab / 50, train_pred_lab / 50, test_lab / 50, test_pred_lab / 50, 9)
    pd.DataFrame(sd_mean).to_csv('sd_mean_' + str(avg_num) + '.csv', index=False, header=False)

'''
##################################光谱平均处理#####################################################
avg_step = 100
# all_feat=dpp.spec_avg_normal(all_feat,avg_step) #####平均光谱的时候使用
##################################光谱归一化及特征选择#####################################################
all_feat_normalspec = dpp.normal_spec(all_feat)
scores = dpp.score(all_feat_normalspec, all_lab)
num_feat = 150
index_feat = dpp.select_best(scores, num_feat)
select_feat = all_feat_normalspec[:, index_feat]
print('光谱归一化及特征选择完成')

##################################数据集划分及包装#####################################################
batch_size = 4000
index_test = []
train_feat, train_lab, test_feat, test_lab = tf.stand_divid(select_feat, all_lab, 9, index_test)
train_feat = tf.a2t(train_feat)
train_lab_tensor = tf.a2t(train_lab)
test_feat = tf.a2t(test_feat)
test_lab_tensor = tf.a2t(test_lab)
train_data = tf.load_array((train_feat, train_lab_tensor), batch_size)
##################################模型训练#####################################################
model = tf.bpnn()
print('测试标签：%f的模型训练开始' % (test_lab[0]))
model = tf.model_train(model, train_data, train_feat, train_lab, test_feat, test_lab)
torch.save(model, 'model_' + str(test_lab[0]) + '.pth')
print('测试标签：%f的模型训练完成' % (test_lab[0]))
##################################模型性能评估#####################################################
'''
